Одной из основных проблем метода Ньютона является то, что он метод 2го порядка, а значит требует вычисления Гессиана, что занимает очень много времени. Одной из оптимизаций может послужить обновление Гессиана лишь раз в несколько итераций, но вычисление обратной матрицы тоже очень затратная операция. Чтобы избежать этих затрат, были изобретены квазиньютоновские методы, они не вычисляют Гессиан на каждой итерации, а лишь приблизительно оценивают вид его обратно матрицы, благодаря чему ассимптотика итерации становится не $O(d^3)$, а $O(d^2)$ где d - размерность функции. Для реализации мы выбрали алгоритм Бройдена — Флетчера — Гольдфарба — Шанно (BFGS). Главной его особенностью является вычисление обратной матрицы Гессианы с помощью рекурентной формулы: $C_{k+1} = (I - r * s * y^T)C_k(I - r * y - s^T) + p * s * s^T$, где 
$r = \frac{1}{y^T*s}$, $y = x_{new} - x$, $s = grad(x_{new}) - grad(x)$

In [ ]:
import sys
sys.path.insert(0, './lib')
from bfgs import BFGS
from newton import NewtonCG
from lrs import gradient, hessian, constant
from function_wrapper import FunctionWrapper
from output import pretty_print
from graphics_plotter import GraphicsPlotter

In [ ]:
func = lambda: FunctionWrapper(lambda x: 3 * x[0]**2 + x[1]**2 - x[1] + 2)

bounds = [[-10, 15], [-10, 15]]

bfgs = BFGS(func(), bounds)
newton = NewtonCG(constant(1), func(), bounds)

x = [-2, -7]
steps = 1000

gradient.clear()
res1 = bfgs.find_min(x, steps)
pretty_print(bfgs, "BFGS", res1, gradient)

hessian.clear()
gradient.clear()
res2 = newton.find_min(x, steps)
pretty_print(newton, "NewtonCG", res2, gradient, hessian)

plotter1 = GraphicsPlotter(bfgs)
plotter2 = GraphicsPlotter(newton)
plotter1.plot()
plotter2.plot()

Данный тест показывает, что метод BFGS делает не отстаёт в точности от метода Ньютона, при этом не обращая Гессиан, а значит каждая итерация требует $O(dim^2)$ действий, да, в случае функций 2 переменных, константа такая, что BFGS делает больше операций, но посмотрим как ведёт себя данный метод на других функциях.

In [ ]:
func = lambda: FunctionWrapper(lambda x: 3 * x[0]**2 + x[1]**4 - x[1] + 2)

bounds = [[-10, 15], [-10, 15]]

bfgs = BFGS(func(), bounds)
newton = NewtonCG(constant(1), func(), bounds)

x = [-2, -7]
steps = 1000

gradient.clear()
res1 = bfgs.find_min(x, steps)
pretty_print(bfgs, "BFGS", res1, gradient)

hessian.clear()
gradient.clear()
res2 = newton.find_min(x, steps)
pretty_print(newton, "NewtonCG", res2, gradient, hessian)

plotter1 = GraphicsPlotter(bfgs)
plotter2 = GraphicsPlotter(newton)
plotter1.plot()
plotter2.plot()

В данном тесте уже BFGS сделал меньше итераций, чем метод Ньютона, при этом, каждая итерация была в 2 раза быстрее, что уже делает BFGS гораздо эффективнее, при этом абсолютно не уступающим в точности.

In [ ]:
func = lambda: FunctionWrapper(lambda x: x[0]**4 + (x[1] - 1)**2 + (x[2] - 2)**2 + (x[3] - 3)**2 + (x[4] - 4)**2)

dim = 5

bounds = [[-10, 15] for i in range(dim)]

bfgs = BFGS(func(), bounds, 1e-3)
newton = NewtonCG(constant(1), func(), bounds)

x = [-2 for i in range(dim)]
steps = 1000

gradient.clear()
res1 = bfgs.find_min(x, steps)
pretty_print(bfgs, "BFGS", res1, gradient)

hessian.clear()
gradient.clear()
res2 = newton.find_min(x, steps)
pretty_print(newton, "NewtonCG", res2, gradient, hessian)

Ну а этот тест окончательно уничтожает метод Ньютона, чётко доказывая, что BFGS работает гораздо эффективнее и быстрее, т.к. метод Ньтона сделал только $13 * 5^3 = 1 625$ действий, в то время как BFGS сделал только $11 * 5^2=275$ что в более чем 5 раз быстрее. А теперь представим, что размерность нашей функции > 1000, и всё станет окончательно понятно.

In [ ]:
from gradient_descent import ScipyWrapper

func = lambda: FunctionWrapper(lambda x: 3 * (x[0] - 3)**6 + x[1]**4 - x[1] + 2)

bounds = [[-10, 15], [-10, 15]]

bfgs = BFGS(func(), bounds)
scipy = ScipyWrapper(func(), bounds)

x = [-2, -7]
steps = 1000

gradient.clear()
res1 = bfgs.find_min(x, steps)
pretty_print(bfgs, "BFGS", res1, gradient)

gradient.clear()
res2 = scipy.find_min("BFGS", x, steps)
pretty_print(scipy, "scipy", res2, gradient)

plotter1 = GraphicsPlotter(bfgs)
plotter2 = GraphicsPlotter(scipy)
plotter1.plot()
plotter2.plot()

Данный тест показывает, что в сравнении с библиотечной реализацией наш метод неплохо справился! А значит, мы написали хорошую реализацию.

**Вывод** Метод Ньютона хороший метод оптимизации, но BFGS буквально его улучшенная версия. Он не проигрывает в точности, однако работает гораздо быстрее и эффективнее засчёт рекурентного подсчёта гессиана. Он отлично подходит для нахождения минимума или максимума на области, на которой один экстремум, и, в отличие от метода Ньютона, имеет ассимтотику $O(dim^2)$, что делает его, пока что, лучшим методом из изученных нами. 